# Dynamic Tiling of STAC assets hosted in AWS

Please note that while this notebook shows HOW we can get data from AWS and do spectral index calculations without having to process and store the geotiff ourselves, this notebook does not currently include any kind of cloud masking.

Cloud masking is needed for actually using these indices, as otherwise the pixels with cloud data mess up the 'valid' pixels. There is a cloud masking function in some of the other notebooks that are targeted to spectral indices.

## To do:
- Add cloud masking before index calculation
- rio-tiler also works with xarray, which means we should be able to use it with `odc-stac` to make single epoch and composite images. (meaning we can replicate EOS)
- Currently using the developmentseed titiler endpoint. Need to get our own titiler endpoint working in the dev container instead.

In [78]:
# Uncomment this line if you need to install the dependencies
%pip install folium requests rasterio pystac-client -q

Note: you may need to restart the kernel to use updated packages.


In [79]:
import os
import httpx
import json
import datetime
import pprint as pp
import urllib.parse
import itertools
import pystac_client

from rio_tiler.io import STACReader
from rio_tiler.models import ImageData

from rasterio.features import bounds as featureBounds

from folium import Map, TileLayer, GeoJson

In [80]:
def initialize_stac_client(stac_url):
    """
    Initialize and return a STAC client for a given STAC API URL.
    Parameters:
    - stac_url (str): The URL of the STAC API.
    Returns:
    - A pystac_client.Client object
    """
    client = pystac_client.Client.open(stac_url)
    return client


def query_stac_api(client, collections, bbox, start_date=None, end_date=None, max_items=100):
    """
    Query a STAC API for items within a bounding box and date range for specific collections.
    Parameters:
    - client: The STAC client initialized with `initialize_stac_client`.
    - bbox (list): The bounding box for the query [min_lon, min_lat, max_lon, max_lat].
    - collections (list): A list of collection IDs to include in the query.
    - start_date (str, optional): The start date for the query (YYYY-MM-DD). Defaults to None.
    - end_date (str, optional): The end date for the query (YYYY-MM-DD). Defaults to None.
    - limit (int): Maximum number of items to return.    
    Returns:
    - A list of STAC Items that match the query parameters.
    """

    search_params = {
        "bbox": bbox,
        "collections": [collections],
        "max_items": max_items
    }
    if start_date and end_date:
        search_params["datetime"] = f"{start_date}/{end_date}"

    search = client.search(**search_params).item_collection()
    return search

In [81]:
titiler_endpoint = "https://titiler.xyz"  # Developmentseed Demo endpoint. Please be kind.
stac_endpoint = "https://earth-search.aws.element84.com/v1"

collection_sentinel = "sentinel-2-l2a"

start_date = "2024-05-20"
end_date = "2024-05-28"
sentinel_bands = ['blue','green','red','nir', 'rededge1', 'swir16', 'scl']

In [82]:
geojson = {
    'body': {
        "type": "FeatureCollection",
        "name": "dissolved-boundaries",
        "crs": {
            "type": "name",
            "properties": {
                "name": "urn:ogc:def:crs:OGC:1.3:CRS84" 
            }
        },
        "features": [
            {
                "type": "Feature",
                "properties": {
                    "fid": 1
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [116.26012130269045, -29.225295369642396],
                            [116.261724812149055, -29.241374854584375],
                            [116.283751968396274, -29.256813692452539],
                            [116.284342735038919, -29.268250184258388],
                            [116.292247755352392, -29.265992437426529],
                            [116.292360282331941, -29.293057573630019],
                            [116.314865678242256, -29.293523728033122],
                            [116.326259034921833, -29.293033039128805],
                            [116.326315298411629, -29.305397680579894],
                            [116.355065941687045, -29.307016748931797],
                            [116.355065941687045, -29.306575187382712],
                            [116.383366477044206, -29.307384715430175],
                            [116.384322956370426, -29.290407813444993],
                            [116.387586238777402, -29.282629879611861],
                            [116.386517232471661, -29.259807919053017],
                            [116.359201308185533, -29.259488866292969],
                            [116.359229439930417, -29.259243440415627],
                            [116.35242155766754, -29.259292525638209],
                            [116.352140240218716, -29.220237788279107],
                            [116.302234524787593, -29.223503148505326],
                            [116.281388901825679, -29.2239696200396],
                            [116.26012130269045, -29.225295369642396]
                        ]
                    ]
                }
            }
        ]
    }
}

req = geojson
geojson_data = req['body'] 

bounds = featureBounds(geojson_data)
print(bounds)

(116.26012130269045, -29.307384715430175, 116.3875862387774, -29.220237788279107)


In [83]:
# check polygon on map before proceeding
m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)

geo_json = GeoJson(data=geojson_data)
geo_json.add_to(m)
m

In [84]:
# STAC query to get items

client_sentinel = initialize_stac_client(stac_endpoint)
items_sentinel = query_stac_api(client_sentinel, collection_sentinel, bounds, start_date, end_date)


In [85]:
items_sentinel

pystac item [options](https://pystac.readthedocs.io/en/latest/api/pystac.html#pystac.Item)

In [86]:
for item in items_sentinel:
    print(item.id)
    pp.pp(item.links[0])
    pp.pp(item.assets)
    pp.pp(item.geometry)
    print()

S2B_50JMN_20240525_0_L2A
<Link rel=self target=https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2B_50JMN_20240525_0_L2A>
{'aot': <Asset href=https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/50/J/MN/2024/5/S2B_50JMN_20240525_0_L2A/AOT.tif>,
 'blue': <Asset href=https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/50/J/MN/2024/5/S2B_50JMN_20240525_0_L2A/B02.tif>,
 'coastal': <Asset href=https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/50/J/MN/2024/5/S2B_50JMN_20240525_0_L2A/B01.tif>,
 'granule_metadata': <Asset href=https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/50/J/MN/2024/5/S2B_50JMN_20240525_0_L2A/granule_metadata.xml>,
 'green': <Asset href=https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/50/J/MN/2024/5/S2B_50JMN_20240525_0_L2A/B03.tif>,
 'nir': <Asset href=https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/50/J/MN/2024/5/S2B_50JMN_2024

In [87]:
bounds = featureBounds(item)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=8
)

geo_json = GeoJson(data=item)
geo_json.add_to(m)
m

In [88]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/info",
    params = (
        ("url", item.links[0].href),
        # Get info for multiple assets
        ("assets","visual"), ("assets","red"), ("assets","blue"), ("assets","green"),
    )
).json()
pp.pp(r)

{'visual': {'bounds': [115.96366797192765,
                       -29.91872967517731,
                       117.10111240379487,
                       -28.923829500519567],
            'minzoom': 8,
            'maxzoom': 14,
            'band_metadata': [['b1', {}], ['b2', {}], ['b3', {}]],
            'band_descriptions': [['b1', ''], ['b2', ''], ['b3', '']],
            'dtype': 'uint8',
            'nodata_type': 'Nodata',
            'colorinterp': ['red', 'green', 'blue'],
            'scales': [1.0, 1.0, 1.0],
            'offsets': [0.0, 0.0, 0.0],
            'driver': 'GTiff',
            'count': 3,
            'width': 10980,
            'height': 10980,
            'overviews': [2, 4, 8, 16],
            'nodata_value': 0.0},
 'red': {'bounds': [115.96366797192765,
                    -29.91872967517731,
                    117.10111240379487,
                    -28.923829500519567],
         'minzoom': 8,
         'maxzoom': 14,
         'band_metadata': [['b1', {}]],
 

### Display one asset

In [89]:
r = httpx.get(
    f"{titiler_endpoint}/stac/WebMercatorQuad/tilejson.json",
    params = {
        "url": item.links[0].href,
        "assets": "visual",
        "minzoom": 8,  # By default titiler will use 0
        "maxzoom": 14, # By default titiler will use 24
    }
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

In [91]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/WebMercatorQuad/tilejson.json",
    params = (
        ("url", item.links[0].href),
        ("assets", "red"),
        ("assets", "green"),
        ("assets", "blue"),
        ("minzoom", 8),
        ("maxzoom", 14),
        ("rescale", "0,2000"),
    )
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=11
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

Use an expression to calculate a band index (NDVI) based on information contained in multiple assets.

In [92]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/WebMercatorQuad/tilejson.json",
    params = (
        ("url", item.links[0].href),
        ("expression", "(nir-red)/(nir+red)"),  # NDVI
        # We need to tell rio-tiler that each asset is a Band
        # (so it will select the first band within each asset automatically)
        ("asset_as_band", True),
        ("rescale", "-1,1"),
        ("minzoom", 8),
        ("maxzoom", 14),
        ("colormap_name", "viridis"),
    )
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

If you don't use the  `asset_as_band=True` option, you need to append the band to the asset name within the expression. For example, `nir` becomes `nir_b1`.

In [93]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/WebMercatorQuad/tilejson.json",
    params = (
        ("url", item.links[0].href),
        ("expression", "(nir_b1-red_b1)/(nir_b1+red_b1)"),  # NDVI
        ("rescale", "-1,1"),
        ("minzoom", 8),
        ("maxzoom", 14),
        ("colormap_name", "viridis"),
    )
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m